# Test ray with pytorch

In [1]:
!whoami

root


In [2]:
!cat /etc/os-release

PRETTY_NAME="Ubuntu 22.04.4 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.4 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy


In [3]:
!python3 --version

Python 3.10.12


In [4]:
!pip show numpy

Name: numpy
Version: 2.2.6
Summary: Fundamental package for array computing in Python
Home-page: https://numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: Copyright (c) 2005-2024, NumPy Developers.
         All rights reserved.
         
         Redistribution and use in source and binary forms, with or without
         modification, are permitted provided that the following conditions are
         met:
         
             * Redistributions of source code must retain the above copyright
                notice, this list of conditions and the following disclaimer.
         
             * Redistributions in binary form must reproduce the above
                copyright notice, this list of conditions and the following
                disclaimer in the documentation and/or other materials provided
                with the distribution.
         
             * Neither the name of the NumPy Developers nor the names of any
                contributors may be used to e

In [5]:
!pip show ray

Name: ray
Version: 2.47.1
Summary: Ray provides a simple, universal API for building distributed applications.
Home-page: https://github.com/ray-project/ray
Author: Ray Team
Author-email: ray-dev@googlegroups.com
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: click, filelock, jsonschema, msgpack, packaging, protobuf, pyyaml, requests
Required-by: 


In [6]:
!nvidia-smi

Thu Jul  3 14:22:04 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.02              Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060        On  |   00000000:01:00.0  On |                  N/A |
|  0%   45C    P8             N/A /  115W |     816MiB /   8188MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [7]:
import torch

# Check PyTorch version
print("PyTorch version:", torch.__version__)

# Check if CUDA is available
print("CUDA available:", torch.cuda.is_available())

# Check CUDA device count
print("Number of GPUs Available:", torch.cuda.device_count())

# Get current GPU device name
if torch.cuda.is_available():
    print("Current GPU:", torch.cuda.get_device_name(torch.cuda.current_device()))

# Optional: Run a simple tensor operation on GPU
def test_gpu():
    if torch.cuda.is_available():
        x = torch.rand(3, 3).cuda()
        print("\nTensor on GPU:", x)
        print("Device of tensor:", x.device)
    else:
        print("No GPU found.")

test_gpu()

PyTorch version: 2.7.1+cu126
CUDA available: True
Number of GPUs Available: 1
Current GPU: NVIDIA GeForce RTX 4060

Tensor on GPU: tensor([[0.2457, 0.9986, 0.7938],
        [0.6558, 0.0366, 0.3220],
        [0.5368, 0.1399, 0.0913]], device='cuda:0')
Device of tensor: cuda:0


In [9]:
!pip show ray

Name: ray
Version: 2.47.1
Summary: Ray provides a simple, universal API for building distributed applications.
Home-page: https://github.com/ray-project/ray
Author: Ray Team
Author-email: ray-dev@googlegroups.com
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: click, filelock, jsonschema, msgpack, packaging, protobuf, pyyaml, requests
Required-by: 


In [8]:
import ray
from ray.rllib.algorithms.ppo import PPOConfig

ray.shutdown()
ray.init()

config = (
    PPOConfig()
    .environment("CartPole-v1")
    .framework("torch")
    .resources(num_gpus=1)  # GPU for learner instead
    .learners(num_learners=1, num_gpus_per_learner=1)
    .api_stack(enable_rl_module_and_learner=True,
               enable_env_runner_and_connector_v2=True)
)
algo = config.build()
for _ in range(3):
    result = algo.train()
    # print("iter:", result["iteration"], "reward:", result["episode_reward_mean"])
    print(result)

ray.shutdown()

2025-07-03 14:22:05,960	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-07-03 14:22:06,274	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-07-03 14:22:07,202	WARNING services.py:2152 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=4.83gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2025-07-03 14:22:07,345	INFO worker.py:1917 -- Started a local Ray instance.
2025-07-03 14:22:07,794	WARNING deprecation.py:50 -- DeprecationWarning: `build` has been dep

{'timers': {'training_iteration': 8.91520766299982, 'restore_env_runners': 1.0882999958994333e-05, 'training_step': 8.915015130000029, 'env_runner_sampling_timer': 1.002568570999756, 'learner_update_timer': 7.904735869999968, 'synch_weights': 0.006697662000078708}, 'env_runners': {'module_to_env_connector': {'connector_pipeline_timer': np.float64(0.00021311385209466362), 'timers': {'connectors': {'remove_single_ts_time_rank_from_batch': np.float64(1.0292252828944501e-06), 'tensor_to_numpy': np.float64(2.9498596923245155e-05), 'normalize_and_clip_actions': np.float64(1.593305192695984e-05), 'get_actions': np.float64(9.213061925904117e-05), 'un_batch_to_individual_items': np.float64(1.0073950039077247e-05), 'listify_data_for_vector_env': np.float64(1.7705940040988167e-05)}}}, 'weights_seq_no': 0.0, 'sample': np.float64(0.9763932645005298), 'num_episodes_lifetime': 174.0, 'episode_len_max': 98, 'env_to_module_sum_episodes_length_out': np.float64(12.060099138522158), 'num_episodes': 174.0,

(_WrappedExecutable pid=2338) 2025-07-03 14:22:11,245	WARNING deprecation.py:50 -- DeprecationWarning: `RLModule(config=[RLModuleConfig object])` has been deprecated. Use `RLModule(observation_space=.., action_space=.., inference_only=.., model_config=.., catalog_class=..)` instead. This will raise an error in the future! [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
